# Build a cuisine recommender

In [1]:
%pip install skl2onnx
import pandas as pd

data = pd.read_csv('../data/cleaned_cuisines.csv')

data.head()

Note: you may need to restart the kernel to use updated packages.


,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [2]:
X = data.iloc[:, 2:]

X.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
y = data[['cuisine']]

y.head()

,cuisine
0,indian
1,indian
2,indian
3,indian
4,indian


## Build the model

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, precision_score
)
import numpy as np

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = SVC(kernel='linear', C=10, random_state=0, probability=True)
model.fit(X_train, np.ravel(y_train))

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     chinese       0.70      0.71      0.71       268
      indian       0.86      0.81      0.84       242
    japanese       0.75      0.70      0.73       235
      korean       0.82      0.79      0.81       237
        thai       0.70      0.81      0.75       217

    accuracy                           0.76      1199
   macro avg       0.77      0.77      0.77      1199
weighted avg       0.77      0.76      0.77      1199



## Convert the model to Onnx

In [7]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}

onnx = convert_sklearn(model, initial_types=initial_type, options=options)
with open('./model.onnx', 'wb') as f:
    f.write(onnx.SerializeToString())